In [1]:
#@title ## Setup Environment { display-mode: "form" }
#@markdown This line will install **pytorch3d**.
!pip install pytorch3d

     |████████████████████████████████| 30.0 MB 100 kB/s 
     |████████████████████████████████| 49 kB 6.7 MB/s 
     |████████████████████████████████| 636 kB 52.0 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20210730-py3-none-any.whl size=60619 sha256=d162ca7faa0d4cee543c6c4f6b4929c5847338b2057fa152764306f1138681ef
  Stored in directory: /root/.cache/pip/wheels/fc/9c/12/9e9331cff3ddb5bb02a818886f0b52f5d160c54efa19d3516a
Successfully built fvcore
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
#@title Mount Drive { display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#@title  Clone repositories { display-mode: "form" }
#@markdown Paste the **path** to your working directory below:

#@markdown (*The path **must** end with **/** for the code to work properly.*)


path = '/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/' #@param {type:'string'}
%cd {path}
print('Navigating to Workdirectory...')
print('Creating "PifuHD"')
!mkdir PifuHD
print('Open PifuHD...')
%cd {path}'PifuHD/'
print('Cloning PifuHD...')
!git clone https://github.com/facebookresearch/pifuhd
print('Cloning lightweight-human-pose-estimation...')
!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git
print('All done!')

/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground
Navigating to Workdirectory...
Creating "PifuHD"
Open PifuHD...
/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/PifuHD
Cloning PifuHD...
Cloning into 'pifuhd'...
remote: Enumerating objects: 213, done.
remote: Total 213 (delta 0), reused 0 (delta 0), pack-reused 213
Receiving objects: 100% (213/213), 402.72 KiB | 1.62 MiB/s, done.
Resolving deltas: 100% (104/104), done.
Cloning lightweight-human-pose-estimation...
Cloning into 'lightweight-human-pose-estimation.pytorch'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 120 (delta 1), reused 0 (delta 0), pack-reused 115
Receiving objects: 100% (120/120), 227.79 KiB | 971.00 KiB/s, done.
Resolving deltas: 100% (50/50), done.
All done!


In [4]:
#@title ## Upload Image { display-mode: "form" }
#@markdown  Run this cell to **upload your image**. Currently PNG, JPEG files are supported. 

%cd {path}'PifuHD/pifuhd/'
!mkdir images
%cd {path}'PifuHD/pifuhd/images'
from google.colab import files
filename = list(files.upload().keys())[0]

/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/PifuHD/pifuhd
/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/PifuHD/pifuhd/images


Saving Beyoncé_background_removed.png to Beyoncé_background_removed.png


In [5]:
#@title Set input and output paths { display-mode: "form"}
import os

try:
  image_path = path +'PifuHD/pifuhd/images/%s' % filename
except:
  image_path = path +'PifuHD/pifuhd/sample_images/test.png' # example image
image_dir = os.path.dirname(image_path)
file_name = os.path.splitext(os.path.basename(image_path))[0]

# output pathes
obj_path = path +'PifuHD/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name
out_img_path = path +'PifuHD/pifuhd/results/pifuhd_final/recon/result_%s_256.png' % file_name
video_path = path +'PifuHD/pifuhd/results/pifuhd_final/recon/result_%s_256.mp4' % file_name
video_display_path = path +'PifuHD/pifuhd/results/pifuhd_final/result_%s_256_display.mp4' % file_name

In [6]:
#@title Load pretrained pose estimation model {display-mode: "form"}
%cd {path}'PifuHD/lightweight-human-pose-estimation.pytorch/'
!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth

/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/PifuHD/lightweight-human-pose-estimation.pytorch
--2021-07-31 09:47:08--  https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth
Resolving download.01.org (download.01.org)... 104.65.185.105, 2600:1406:3c:397::4b21, 2600:1406:3c:38e::4b21
Connecting to download.01.org (download.01.org)|104.65.185.105|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87959810 (84M) [application/octet-stream]
Saving to: ‘checkpoint_iter_370000.pth’

checkpoint_iter_370 100%[===================>]  83.88M  59.1MB/s    in 1.4s    

2021-07-31 09:47:10 (59.1 MB/s) - ‘checkpoint_iter_370000.pth’ saved [87959810/87959810]



In [7]:
#@title Run PoseEstimation {display-mode: "form"}
#@markdown In this cell the image gets optimized and then the pose is estimated. Note that it only partially works if the legs are missing.

import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo

#-----Configurations Start ----# 

##Only Change these if you know what you are doing!

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)
            
            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

#-----Configurations End ----#


net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)

get_rect(net.cuda(), [image_path], 512)


In [8]:
#@title Convert to 3D Model {display-mode: "form"}
print('Downloading pretrained model...')
print(' ')

%cd {path}'PifuHD/pifuhd/'
!sh ./scripts/download_trained_model.sh
print(' ')
print('Download Completed.')
print(' ')
print('Converting Image to 3D-Obj...')
# Warning: all images with the corresponding rectangle files under -i will be processed. 
!python -m apps.simple_test -r 256 --use_rect -i $image_dir
print(' ')
print('All done!')
print(' ')
print('You can find your results here:')
print(obj_path)

# seems that 256 is the maximum resolution that can fit into Google Colab. 
# If you want to reconstruct a higher-resolution mesh, please try with your own machine. 

 
/content/drive/MyDrive/AiXDesignKitchen/Workshop/Playground/PifuHD/pifuhd
+ mkdir -p checkpoints
+ cd checkpoints
+ wget https://dl.fbaipublicfiles.com/pifuhd/checkpoints/pifuhd.pt pifuhd.pt
--2021-07-31 09:47:32--  https://dl.fbaipublicfiles.com/pifuhd/checkpoints/pifuhd.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1548375177 (1.4G) [application/octet-stream]
Saving to: ‘pifuhd.pt’

pifuhd.pt           100%[===================>]   1.44G  58.3MB/s    in 27s     

2021-07-31 09:47:59 (54.5 MB/s) - ‘pifuhd.pt’ saved [1548375177/1548375177]

--2021-07-31 09:47:59--  http://pifuhd.pt/
Resolving pifuhd.pt (pifuhd.pt)... failed: Name or service not known.
wget: unable to resolve host address ‘pifuhd.pt’
FINISHED --2021-07-31 09:47:59--
Total wall clock time: 27s
Downloaded: 1 fi